In [1]:
!pip install tensorflow==1.15.0
!pip install deeppavlov
!pip install spacy<2.2
!python -m deeppavlov install squad_bert
#!pip install pytorch-crf
#!pip install pandas==1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 27 kB/s 
     |████████████████████████████████| 3.8 MB 29.0 MB/s 
     |████████████████████████████████| 503 kB 61.8 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7555 sha256=960bd0bde9f1727844e7aa83cfe49d3f56e5766a8854b8210d0d2ccb070c96a2
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting un

**Importante**

- Python 3.7

- tensorflow 1.15.2



In [5]:
from deeppavlov import configs, build_model
import warnings
warnings.filterwarnings('ignore')
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)


2022-11-08 20:48:42.329 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_mult_torch_crf.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_mult_torch_crf.tar.gz download because of matching hashes
2022-11-08 20:48:42.787 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_torch_bert_mult_crf/tag.dict]
INFO:deeppavlov.core.data.simple_vocab:[loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_torch_bert_mult_crf/tag.dict]


Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [7]:
import pandas as pd

In [8]:
df_tweets = pd.read_csv("https://raw.githubusercontent.com/Izainea/pruebas_vivi/master/prueba_NLP_cientifico_vivi/Datos/davivienda_tweets.csv")

In [14]:
#df_tweets['Embedded_text']

In [33]:
TextoNER_1=[]
Entidad=[]
for i in df_tweets['Embedded_text']:
    TextoNER_1.append(ner_model([i])[0][0])
    Entidad.append(ner_model([i])[1][0])
TextoNER_1[0]

['La',
 'confianza',
 'se',
 'afectó',
 '.',
 'El',
 'indicador',
 'de',
 'confianza',
 'Davivienda',
 'tuvo',
 'una',
 'leve',
 'caída',
 'en',
 'noviembre',
 ',',
 'rompiendo',
 'una',
 'tendencia',
 'de',
 'cinco',
 'meses',
 'de',
 'mejoras',
 '.',
 'El',
 'indicador',
 'se',
 'deterioró',
 ',',
 'especialmente',
 'en',
 'la',
 'última',
 'semana',
 'del',
 'mes',
 ',',
 'asociado',
 'al',
 'aumento',
 'en',
 'la',
 'tasa',
 'de',
 'cambio',
 '.',
 '\n',
 '1',
 '\n',
 '7',
 '\n',
 '19']

In [39]:
def facilent(Ent):
    tokens, tags = Ent
    text=[]
    types=[]
    start=[]
    end=[]
    svalue=0
    csvalue=0
    cevalue=0
    for p in range(len(tags[0])):
        cevalue=csvalue+len(tokens[0][p])
        
        if tags[0][p][0]=='B':
            svalue=p 
            start.append(csvalue)
            types.append("".join(Entidad[p][2:]))
            if tags[0][p+1][0]=='B':
                end.append(cevalue)
                text.append(" ".join(tokens[0][svalue:p+1]))
            elif tags[0][p+1][0]=='O':
                end.append(cevalue)
                text.append(" ".join(tokens[0][svalue:p+1]))

        elif tags[0][p][0]=='I':
            if tags[0][p+1][0]=='I':
                None
            else:
                end.append(cevalue)
                text.append(" ".join(tokens[0][svalue:p+1]))
        csvalue=cevalue+1
    
    Ents=[]
    for i in range(len(types)):
        Ents.append({"text":text[i],"type":types[i],"start_chart":start[i],"end_chart":end[i]})
    return Ents


def displayent(TextoNer,Entidad):
    TextoNer=TextoNer+['pala']
    Entidad = Entidad+['O']
    text=[]
    types=[]
    start=[]
    end=[]
    svalue=0
    csvalue=0
    cevalue=0
    for p in range(len(Entidad)-1):
        cevalue=csvalue+len(TextoNer[p])
        
        if Entidad[p][0]=='B':
            svalue=p 
            start.append(csvalue)
            types.append("".join(Entidad[p][2:]))
            if Entidad[p+1][0]=='B':
                end.append(cevalue)
                text.append(" ".join(TextoNer[svalue:p+1]))
            elif Entidad[p+1][0]=='O':
                end.append(cevalue)
                text.append(" ".join(TextoNer[svalue:p+1]))

        elif Entidad[p][0]=='I':
            if Entidad[p+1][0]=='I':
                None
            else:
                end.append(cevalue)
                text.append(" ".join(TextoNer[svalue:p+1]))
        csvalue=cevalue+1
    
    ents=[]
    for i in range(len(types)):
        ents.append({"label":types[i],"start":start[i],"end":end[i]})
    Ents=[{"text":" ".join(TextoNer),"ents":ents,"tilte":None}]
    return Ents



In [42]:
Lista_display_ent=[]
for i in range(1811):
    de=displayent(TextoNER_1[i],Entidad[i])
    Lista_display_ent.append(de)

In [44]:
from spacy import displacy
import numpy as np 
for i in np.random.randint(0,1811,10):
  html = displacy.render(displayent(TextoNER_1[i],Entidad[i]), style="ent", manual=True)
  from IPython.core.display import display,HTML
  display(HTML(html))

In [48]:
list_df_entities=[]
for k in range(1811):
  Texto=[]
  Ent=[]
  for i in displayent(TextoNER_1[k],Entidad[k])[0]['ents']:
      Texto.append(displayent(TextoNER_1[k],Entidad[k])[0]['text'][i['start']-1:i['end']+1])
      Ent.append(i['label'])
  DF_temp=pd.DataFrame([Texto,Ent]).T
  DF_temp.columns=['Texto','Ent']
  DF_temp['tweet_id']=k
  list_df_entities.append(DF_temp)

DF_entities=pd.concat(list_df_entities)
DF_entities.to_csv('Tweets_entidades.csv')  
  

In [49]:
DF_entities

,Texto,Ent,tweet_id
0,Davivienda,ORG,0
1,noviembre,DATE,0
2,cinco meses,DATE,0
3,la última semana del mes,DATE,0
4,1,CARDINAL,0
...,...,...,...
0,Hola Jose,PERSON,1807
0,Karen,PERSON,1808
0,Buenos días,DATE,1809
1,Diego,PERSON,1809
